# IMPORTING THE REQUIREMENTS

In [41]:
import os
import tempfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

# LINKING TO THE DIRECTORY

In [34]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [35]:
%cd /content/drive/My Drive/CovidDataset

/content/drive/My Drive/CovidDataset


In [37]:
!ls

covid19.jpeg  Covid_xray_model.h5  normal_1.jpg  normal.jpg  Train  Validation


# LOADING MODEL

In [28]:
model = load_model("Covid_xray_model.h5")

# SAMPLE PREDICTION

In [39]:
img = image.load_img("Validation/Normal/NORMAL2-IM-0601-0001.jpeg", target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

pred = model.predict_classes(img)

print(pred[0, 0])

if pred == 1:
  print("Normal")
elif pred == 0:
  print("Covid")

1
Normal


# SAVING THE ENTIRE MODEL TO PROTOBUF FORMAT 

In [42]:
MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /tmp/1/assets

Saved model:
total 228
drwxr-xr-x 2 root root   4096 Jul 31 18:08 assets
-rw-r--r-- 1 root root 222809 Jul 31 18:08 saved_model.pb
drwxr-xr-x 2 root root   4096 Jul 31 18:08 variables


# EXAMINING THE MODEL

In [43]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['conv2d_8_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 224, 224, 3)
        name: serving_default_conv2d_8_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_5'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0731 18:09:34.146137 140285786208128 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_cor

# ADDING TENSORFLOW SERVING DISTRIBUTION API

In [44]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  18393      0 --:--:-- --:--:-- --:--:-- 18393
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machin

# INSTALLING TENSORFLOW SERVING

In [45]:
! apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 187 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.2.0 [187 MB]
Fetched 187 MB in 3s (68.6 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.2.0_all.deb ...
Unpacking tensorflow-model-server (2.2.0) ...
Setting up tensorflow-model-server (2.2.0) ...


# SERVING THE MODEL

In [46]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [47]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=covid19_Xray_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [49]:
!tail server.log

2020-07-31 18:14:56.209571: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-07-31 18:14:56.230165: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2020-07-31 18:14:56.321864: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:183] Running initialization op on SavedModel bundle at path: /tmp/1
2020-07-31 18:14:56.334054: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:364] SavedModel load for tags { serve }; Status: success: OK. Took 128571 microseconds.
2020-07-31 18:14:56.335802: I tensorflow_serving/servables/tensorflow/saved_model_warmup.cc:105] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2020-07-31 18:14:56.336143: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: covid19_Xray_model version: 1}
2020-07-31 18:14:5

# MAKE A REQUEST TO THE MODEL FOR PREDICTION IN TENSORFLOW SERVING

In [56]:
img = image.load_img("covid19.jpeg", target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

pred = model.predict_classes(img)

print(pred[0, 0])

if pred == 1:
  print("Normal")
elif pred == 0:
  print("Covid")

0
Covid


In [57]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": img.tolist()})

In [73]:
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/covid19_Xray_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

if predictions == [[1.0]]:
  print("Normal")
elif predictions == [[0.0]]:
  print("Covid")

Covid
